[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mravanba/comp551-notebooks/blob/master/EMforGaussianMixture.ipynb)

# Expectation Maximization for Gaussian Mixture Model
In K-means each datapoint is assigned to one cluster. An alternative is for each datapoint ($n$) to have a distribution over clusters -- that is $r_{n,k} \in [0,1]$ and $\sum_k r_{n,k} = 1$. To do this we can assume each cluster has a Gaussian distribution $\mathcal{N}(\mu_k, \Sigma_k)$. So we assume the data-distrubtion is a **mixture of Gaussians**
$$
p(x; \pi, \{\mu_k, \Sigma_k\}) = \sum_k \pi_k \mathcal{N}(x; \mu_k, \Sigma_k)
$$
where $\pi_k= p(z=k)$ with $\sum_k \pi_k = 1$ defining the weight of each Gaussian in the mixture. These weights should sum to one so that we have a valid pdf. To maximize the logarithm of the marginal-likelihood 
$\ell(\pi, \{\mu_k, \Sigma_k\}) = \sum_n \log \left ( \sum_k \pi_k \mathcal{N}(x; \mu_k, \Sigma_k) \right)$, we set its partial derivative wrt various parameters to zero. This gives us the value of these parameters in terms of membership probabilities, aka *responsibilities*: $r_{n,k} = p(z=k|x^{(n)})= \frac{\pi_k \mathcal{N}(x; \mu_k, \Sigma_k)}{\sum_c \pi_c \mathcal{N}(x; \mu_c, \Sigma_c)}$. Since responsibilities are functions of model parmeters, we perform an iterative updating of these two values:
1. update responsibilites given the model parameters 
2. given the responsibilities $r_{n,k}$, update the parameters $\mu_k, \Sigma_k$ and $\pi$. As we said these updates are given by taking the derivative of the log-likelihood:
    - New $\pi_k$ is easy to estimate, it is proportional to the $\sum_n r_{n,k}$. Since $\pi_k$ should sum to one we set
$$
\pi_k = \frac{\sum_n r_{n,k}}{\sum_{n,c} r_{n,c}}
$$
    - For $\mu_k$ and $\Sigma_k$ we need to estimate mean and covariance of a Gaussian using *weighted* samples, where the (unnormalized) weights for the $k^{th}$ Gaussian in the mixture are $r_{n,k} \forall n$.
\begin{align}
\mu_k &= \frac{\sum_n r_{n,k} x^{(n)}}{\sum_n r_{n,k}} \\
\Sigma_k &= \frac{ \sum_n r_{n,k} (x^{(n)} - \mu_k)(x^{(n)} - \mu_k)^\top }{\sum_n r_{n,k}}
\end{align}
The above gives us weighted mean and weighted covariance. We then repeat steps 1 and 2 similar to K-means.

Lets implement EM for Gaussian Mixture Model (GMM) below. We re-use our previous impelementation of multivariate Gaussian in GMM.

In [ ]:
import numpy as np
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import scipy as sp
# Set random seed for reproducibility
# This ensures same initialization across runs
np.random.seed(1234)

In [ ]:
# For more comments on multivariate Gaussian class refer to: 
# https://github.com/mravanba/comp551-notebooks/blob/master/Gaussian.ipynb
class Gaussian():
    """
    Multivariate Gaussian distribution.
    
    Represents a D-dimensional Gaussian with mean mu and covariance Sigma.
    Can evaluate the probability density function at given points.
    """
    def __init__(self, mu=0, sigma=0):
        """
        Initialize a Gaussian distribution.
        
        Parameters:
        mu: mean vector, shape (D,) or scalar
        sigma: covariance matrix (D, D) or variance (scalar)
        """
        # Ensure mu is at least 1D array, even if scalar is passed
        # atleast_1d converts: scalar -> (1,), already 1D -> unchanged
        self.mu = np.atleast_1d(mu)
        
        # Handle sigma based on dimensionality
        if np.array(sigma).ndim == 0:
            # If sigma is scalar (0D), convert to 2D covariance matrix
            # sigma**2 because standard deviation -> variance
            # atleast_2d: scalar -> (1,1) matrix
            self.Sigma = np.atleast_2d(sigma**2)
        else:
            # If sigma is already 2D (covariance matrix), use as is
            self.Sigma = sigma

    def density(self, x):
        """
        Compute probability density at points x.
        
        Parameters:
        x: data points, shape (N, D)
        
        Returns:
        p: density values, shape (N,) - one probability for each point
        """
        N, D = x.shape
        
        # Center the data: subtract mean from each point
        # self.mu has shape (D,)
        # self.mu[None,:] adds a dimension -> (1, D) for broadcasting
        # x has shape (N, D)
        # Broadcasting: (N, D) - (1, D) -> (N, D)
        # Result: xm[i,j] = x[i,j] - mu[j]
        xm = (x - self.mu[None, :])
        
        # Normalization constant for multivariate Gaussian
        # (2π)^(-D/2) * |Σ|^(-1/2)
        # where |Σ| is the determinant of the covariance matrix
        normalization = (2*np.pi)**(-D/2) * np.linalg.det(self.Sigma)**(-1/2)
        
        # Compute Mahalanobis distance: (x-μ)^T Σ^(-1) (x-μ) for each point
        # xm @ np.linalg.inv(self.Sigma) has shape (N, D)
        # Element-wise multiplication with xm, then sum over D dimensions
        # np.sum(..., axis=1) reduces (N, D) -> (N,)
        # Result: quadratic[i] = (x[i] - μ)^T Σ^(-1) (x[i] - μ)
        quadratic = np.sum((xm @ np.linalg.inv(self.Sigma)) * xm, axis=1)
        
        # Gaussian formula: normalization * exp(-0.5 * quadratic_form)
        # Returns (N,) array of density values
        return normalization * np.exp(-.5 * quadratic)


class GMM:
    """
    Gaussian Mixture Model using Expectation-Maximization algorithm.
    
    Fits a mixture of K Gaussian distributions to data by iteratively:
    1. E-step: Computing responsibilities (which Gaussian generated each point)
    2. M-step: Updating Gaussian parameters given responsibilities
    """
    def __init__(self, K=5, max_iters=200, epsilon=1e-5):
        """
        Initialize GMM parameters.
        
        Parameters:
        K: number of Gaussian components in the mixture
        max_iters: maximum number of EM iterations
        epsilon: convergence tolerance and regularization constant
        """
        self.K = K                            # Number of Gaussians
        self.max_iters = max_iters            # Maximum iterations for EM
        self.epsilon = epsilon                # Tolerance for convergence + regularization
    
    def fit(self, x):
        """
        Fit Gaussian Mixture Model to data using EM algorithm.
        
        Parameters:
        x: data points, shape (N, D) where N=samples, D=features
        
        Returns:
        mu: means of K Gaussians, shape (K, D)
        sigma: covariances of K Gaussians, shape (K, D, D)
        r: responsibilities (unnormalized), shape (N, K)
        """
        N, D = x.shape
        
        # ===== INITIALIZATION =====
        
        # Randomly select K data points as initial cluster centers
        # np.random.choice(N, self.K) returns K random integers from [0, N-1]
        # These are indices of the chosen data points
        init_centers = np.random.choice(N, self.K)
        
        # Initialize mixing coefficients π_k (weight of each Gaussian)
        # Start with uniform weights: each Gaussian has weight 1/K
        # Shape: (K,), sums to 1
        pi = np.ones(self.K) / self.K
        
        # Initialize means: use the K randomly selected data points
        # x[init_centers] uses advanced indexing to select rows
        # Shape: (K, D) - each row is the mean for one Gaussian
        mu = x[init_centers]
        
        # Initialize covariance matrices for all K Gaussians
        # Step 1: np.var(x, axis=0) computes variance along each feature
        #         Shape: (D,) - one variance per feature
        # Step 2: np.diag(...) creates diagonal matrix from these variances
        #         Shape: (D, D) - diagonal covariance (assumes features independent)
        # Step 3: [None,:,:] adds a dimension -> (1, D, D)
        # Step 4: np.tile(..., (self.K, 1, 1)) replicates K times along first axis
        #         Shape: (K, D, D) - same diagonal covariance for all K Gaussians
        sigma = np.tile(np.diag(np.var(x, axis=0))[None, :, :], (self.K, 1, 1))
        
        # Initialize responsibilities matrix
        # r[n,k] will store π_k * p(x_n | μ_k, Σ_k)
        # Shape: (N, K) - each row is one data point, each column is one Gaussian
        r = np.zeros((N, self.K))
        
        # Initialize log-likelihood to negative infinity
        # Will be updated in each iteration to check convergence
        ll = -np.inf
        
        # ===== EM ALGORITHM =====
        
        for t in range(self.max_iters):
            # ----- E-STEP: Update responsibilities -----
            
            # For each Gaussian k, compute π_k * p(x | μ_k, Σ_k) for all points
            for i in range(self.K):
                # Gaussian(mu[i], sigma[i]).density(x) computes p(x | μ_i, Σ_i)
                # Shape: (N,) - density for all N points under Gaussian i
                # Multiply by pi[i] (mixing coefficient) to get π_i * p(x | μ_i, Σ_i)
                # r[:,i] assigns to column i (all rows, column i)
                r[:, i] = pi[i] * Gaussian(mu[i], sigma[i]).density(x)
            
            # Normalize responsibilities to get posterior p(z=k | x)
            # np.sum(r, axis=1) sums across Gaussians (axis 1), shape: (N,)
            # keepdims=True preserves dimension: (N,) -> (N, 1)
            # This allows broadcasting: (N, K) / (N, 1) -> (N, K)
            # After normalization: each row sums to 1
            # r_norm[n,k] = π_k * p(x_n|k) / Σ_c π_c * p(x_n|c) = p(z=k | x_n)
            r_norm = r / np.sum(r, axis=1, keepdims=True)

            # ----- M-STEP: Update parameters given responsibilities -----
            
            for i in range(self.K):
                # Update mean μ_k as weighted average of data
                # np.average(x, axis=0, weights=r_norm[:,i])
                # - x has shape (N, D)
                # - weights=r_norm[:,i] has shape (N,) - responsibility for Gaussian i
                # - axis=0 means average over samples (rows)
                # Result: μ_i = Σ_n r_{n,i} * x_n / Σ_n r_{n,i}
                # Shape: (D,) - one mean value per feature
                mu[i, :] = np.average(x, axis=0, weights=r_norm[:, i])
                
                # Update covariance Σ_k as weighted covariance matrix
                # np.cov(x, aweights=r_norm[:,i], rowvar=False)
                # - x has shape (N, D)
                # - aweights=r_norm[:,i] are the analytic weights (responsibilities)
                # - rowvar=False: each column is a variable (not each row)
                # Result: Σ_i = Σ_n r_{n,i} * (x_n - μ_i)(x_n - μ_i)^T / Σ_n r_{n,i}
                # Shape: (D, D)
                # Add ε*I for regularization to prevent singular matrices
                # self.epsilon * np.eye(D) creates diagonal matrix with ε on diagonal
                # [None,:,:] adds dimension: (D,D) -> (1,D,D) for assignment to sigma[i]
                sigma[i, :, :] = np.cov(x, aweights=r_norm[:, i], rowvar=False) + \
                                  self.epsilon * np.eye(D)[None, :, :]
            
            # Update mixing coefficients π_k
            # π_k should be proportional to total responsibility assigned to Gaussian k
            # np.sum(r_norm, axis=0) sums over data points (axis 0)
            # Shape: (K,) - one value per Gaussian
            # Result: π_k ∝ Σ_n r_{n,k}
            pi = np.sum(r_norm, axis=0)
            # Normalize so Σ_k π_k = 1 (valid probability distribution)
            pi /= np.sum(pi)
            
            # ----- CHECK CONVERGENCE -----
            
            # Calculate log-likelihood: log p(x) = log Σ_k π_k p(x|k)
            # np.sum(r, axis=1) computes Σ_k π_k p(x_n|k) for each point, shape (N,)
            # np.log(...) takes logarithm
            # np.mean(...) averages over all data points
            ll_new = np.mean(np.log(np.sum(r, axis=1)))
            
            # Check if change in log-likelihood is below tolerance
            if np.abs(ll_new - ll) < self.epsilon:
                print(f'converged after {t} iterations, average log-likelihood {ll_new}')
                break
            
            # Update log-likelihood for next iteration
            ll = ll_new
        
        return mu, sigma, r

In the implementation above note that we add a diagonal matrix with small constant values ($\epsilon$) on the diagonal to the empirical covariance matrix. This is to avoid degeneracy. If the model puts one of the Gaussians centered on a data-points and make the covariance very small, it can arbitrarily increase the likelihood of the data (see Bishop p.434). This addition of epsilon to the diagonal prevents this degeneracy.

Now let's apply this to the Iris dataset 

In [ ]:
from sklearn import datasets

# Load the famous Iris dataset
dataset = datasets.load_iris()

# Extract features and labels
# dataset['data'] has shape (150, 4) - 150 samples, 4 features
# [:,:2] uses slicing to select only first 2 features (sepal length and width)
# This allows us to visualize in 2D
# x has shape (150, 2)
# y has shape (150,) - contains class labels 0, 1, or 2
x, y = dataset['data'][:, :2], dataset['target']

# Fit Gaussian Mixture Model with K=3 components
gmm = GMM(K=3)
mu, sigma, resp = gmm.fit(x)

# Normalize responsibilities to get posterior probabilities
# resp has shape (N, K) where each entry is π_k * p(x_n|k)
# np.sum(resp, axis=1, keepdims=True) has shape (N, 1)
# After division: each row sums to 1, representing p(z=k|x_n) for each k
resp /= np.sum(resp, axis=1, keepdims=True)

# ===== CREATE VISUALIZATION WITH 3 SUBPLOTS =====

fig, axes = plt.subplots(ncols=3, nrows=1, constrained_layout=True, figsize=(15, 5))

# ----- LEFT PLOT: GMM clustering with contours -----

# Plot data points colored by responsibilities
# resp has shape (N, K), used as RGB colors for scatter plot
# Each point's color is a mix of the K Gaussians it belongs to
# s=2 sets small marker size
axes[0].scatter(x[:, 0], x[:, 1], c=resp, s=2)

# Plot cluster centers (means) with 'x' markers
# mu has shape (K, D) where D=2
# mu[:,0] selects first feature (x-coordinate), shape (K,)
# mu[:,1] selects second feature (y-coordinate), shape (K,)
axes[0].scatter(mu[:, 0], mu[:, 1], marker='x')

# Create a dense grid for visualizing Gaussian density contours
# np.linspace creates evenly spaced values across the data range
# x0v and x1v are 1D arrays of length 200
x0v = np.linspace(np.min(x[:, 0]), np.max(x[:, 0]), 200)
x1v = np.linspace(np.min(x[:, 1]), np.max(x[:, 1]), 200)

# Create 2D meshgrid
# x0 and x1 are both (200, 200) arrays
# x0[i,j] contains the first feature value at grid point (i,j)
# x1[i,j] contains the second feature value at grid point (i,j)
x0, x1 = np.meshgrid(x0v, x1v)

# Flatten and stack to create array of all grid points
# x0.ravel() flattens (200, 200) -> (40000,)
# x1.ravel() flattens (200, 200) -> (40000,)
# np.vstack stacks vertically to create (2, 40000)
# .T transposes to get (40000, 2)
# Each row is a 2D point [x0, x1] in the feature space
x_all = np.vstack((x0.ravel(), x1.ravel())).T

# Get density values for each Gaussian at all grid points
# Gaussian(mu[0], sigma[0]).density(x_all) evaluates first Gaussian
# Returns shape (40000,) - density at each grid point
p0 = Gaussian(mu[0], sigma[0]).density(x_all)
p1 = Gaussian(mu[1], sigma[1]).density(x_all)
p2 = Gaussian(mu[2], sigma[2]).density(x_all)

# Stack densities to create mixture
# np.vstack([p0, p1, p2]) has shape (3, 40000)
# .T transposes to (40000, 3)
# Each row represents densities [p0, p1, p2] at one grid point
p = np.vstack([p0, p1, p2]).T

# Normalize to get responsibilities at each grid point
# np.sum(p, axis=1, keepdims=True) has shape (40000, 1)
# After division: each row sums to 1
# p[i,k] = p_k(grid_point_i) / Σ_j p_j(grid_point_i)
p /= np.sum(p, axis=1, keepdims=True)

# Draw contour lines for each Gaussian's density
# tricontour works with unstructured grids (x_all is flattened)
# x_all[:,0] is first feature (x-coordinates), shape (40000,)
# x_all[:,1] is second feature (y-coordinates), shape (40000,)
# p0, p1, p2 are density values, shape (40000,)
# This draws level curves showing regions of equal probability density
axes[0].tricontour(x_all[:, 0], x_all[:, 1], p0)
axes[0].tricontour(x_all[:, 0], x_all[:, 1], p1)
axes[0].tricontour(x_all[:, 0], x_all[:, 1], p2)
axes[0].set_title('Mixture of Gaussians found using EM')

# ----- MIDDLE PLOT: Cluster responsibilities as continuous colors -----

# Plot training data colored by true labels
# y contains integers 0, 1, 2 representing true classes
axes[1].scatter(x[:, 0], x[:, 1], c=y, s=2)

# Overlay grid points colored by responsibilities
# p is (40000, 3) - RGB-like colors showing cluster membership
# marker='.' creates tiny dots
# alpha=.01 makes points very transparent to show smooth gradient
# This creates a background showing how the model divides the space
axes[1].scatter(x_all[:, 0], x_all[:, 1], c=p, marker='.', alpha=.01)
axes[1].set_title('Cluster Responsibilities')

# ----- RIGHT PLOT: True class labels for comparison -----

# Plot data colored by true labels (no model predictions)
# This shows the ground truth for comparison with model output
axes[2].scatter(x[:, 0], x[:, 1], c=y, s=2)
axes[2].set_title('class labels')

plt.show()

A mixture of Gaussians where the membership of each point is unobserved is an example of a **latent variable model**.
A latent variable model $p(x,z; \theta)$ assumes unobserved or latent variables $z$ that help explain the observations $x$. In this setup we still want to maximize the **marginal likelihood** of data ($z$ is marginalized out):
$$
\max_\theta \sum_n \log p(x^{(n)}; \theta) = \max_\theta \sum_n \log \sum_z p(x^{(n)}, z; \theta)
$$
EM can be used for learning with *latent variable models* or when we have *missing data*. The general approach is similar to what we saw here: 
- computing the posterior $p(z | x^{(n)}; \theta) \forall n$ (Expectation or **E step**)
- maximizing the expected log-likelihood using this probabilistic *completion* of the data (Maximization or **M step**)